In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mstats
from IPython.display import display
from ipython_helpers import (
    print_full
)

In [17]:
def transform_daily_price_data(df):
    """Transforms daily price data to enable making of variables 
    
    The following operations are conducted in this function:
    1) Convert prices to become positive numbers
    2) Convert return column to float type 
    3) Parse the date column to create year and month columns. 
    The year and month columns will be used to group return data 
    4) Make is_non_zero indicator. "is_non_zero" column will be used to count 
    number of active trading days
    """
    df.loc[:, 'RET'] = pd.to_numeric(df['RET'], errors='coerce', downcast='float') #2
    df.loc[:, 'PRC'] = df['PRC'].abs() 
    df.loc[:, 'year'] = df['date'].dt.year
    df.loc[:, 'month'] = df['date'].dt.month
    df.loc[:, 'is_non_zero'] = np.where(df['RET'].abs() > 0, 1, 0)

    return df


def make_variance_df(df):
    """Make a column containing variance for each month
    """
    df = df.groupby(['PERMNO', 'year', 'month']).agg({'RET': 'var', 'is_non_zero': 'sum'})
    df.rename(columns={'RET': 'var'}, inplace=True)
    
    return df.reset_index()
    

def compute_rolling_sum(values):
    """Helper function for make_rolling_columns function
    """
    return np.sum(values[:3])


def make_rolling_columns(df):
    """Helper function for make_rolling_variance_df function
    """
    df = df.sort_values(by=['year', 'month'])
    df['rolling_is_non_zero'] = df['is_non_zero'].rolling(4).apply(compute_rolling_sum)
    df['rolling_var'] = df['var'].rolling(4).apply(compute_rolling_sum)
    
    return df
    
    
def make_rolling_variance_df(df):
    """Make a column containing variance over the rolling 3 months period
    """
    df = df.sort_values(by=['PERMNO', 'year', 'month'])
    return df.groupby(by=['PERMNO']).apply(make_rolling_columns)


def make_mean_df(df):
    """Make a dataframe representing mean of cross sectional return for each month in the data
    
    To eliminate the effect of rolling 3 months period that had few active trading days, 
    observations with fewer than 5 active trading days in the 3 months period will be removed 
    from computing mean of cross sectional return 
    """
    df = df[df['rolling_is_non_zero'] >= 5]
    
    return df.groupby(['year', 'month']).agg({'SIGMA': 'mean'})


def compute_sigma(row):
    """Helper function for make_sigma_df function. 
    """
    if row['rolling_is_non_zero'] >= 5:
        return  np.sqrt(252 / (row['rolling_is_non_zero'] - 1) * row['rolling_var'])
    else:
        # Mark observations with fewer than five nonzero observations as missing
        return np.nan

    
def replace_sigma(row, mean_df):
    """Helper function for make_sigma_df function
    """
    if (pd.isnull(row['SIGMA'])) and (pd.notnull(row['rolling_var'])):
        try:
            return mean_df.loc[row['year'], row['month']]['SIGMA']
        except:
            print(row)
    else:
        return row['SIGMA']

        
def make_sigma_df(df):
    """Make a column containing SIGMA values.
    
    - It computes the SIGMA value for each row by applying the formula mentioned in the 
    Campbell paper. 
    
    - To eliminate the effect of rolling 3 months period that had few active 
    trading days, we will mark those observations with less than 5 active trading days in the 
    3 months period as missing and then replace missing observations with cross sectional mean of SIGMA.

    - To make sure we don't replace SIGMA that were meant to be missing such as the 
    first few observations of each company, we replace this value only if rolling_var 
    exists, which are calculated as long as there were trading days in the rolling window period
    """
    # Compute SIGMA or annualized 3-month rolling sample standard deviation
    df.loc[:, 'SIGMA'] = df.apply(lambda row: compute_sigma(row), axis=1)
    
    # Make dataframe of cross sectional mean of SIGMA
    mean_df = make_mean_df(df)
    
    df.loc[:, 'SIGMA'] = df.apply(lambda row: replace_sigma(row, mean_df), axis=1)
            
    return df


def winsorize_df(df, var_list):
    """Winsorizes the variables specified in the var_list at 5th and 95th percentile
    """
    for var in var_list:
        column_name = var + '_win'
        win_df = df.dropna(subset=[var])
        win_df.loc[:, column_name] = mstats.winsorize(win_df[var], limits=(0.05, 0.05))
        df = df.join(win_df[column_name])
    
    return df 

In [3]:
daily_prices_df = pd.read_csv(
    "../Data/original_data/crsp_daily_equity_1961_2016.csv", 
    usecols=['PERMNO', 'date', 'PRC', 'RET', 'TICKER', 'CUSIP'],
    dtype={
        'PERMNO': np.int32,
        'TICKER': str,         
        'CUSIP': str, 
        'PRC': np.float64,
        'RET': str
    },    
    parse_dates=['date']
)

In [10]:
transformed_df = transform_daily_price_data(daily_prices_df)

In [11]:
variance_df = make_variance_df(transformed_df)

In [16]:
rolling_variance_df = make_rolling_variance_df(variance_df)

In [18]:
SIGMA_df = make_sigma_df(rolling_variance_df)

In [22]:
SIGMA_df1 = SIGMA_df.reset_index(drop=True)

In [23]:
SIGMA_df1.to_csv('../Data/campbell_data/campbell_SIGMA.csv')

In [ ]:
winsorized_SIGMA_df = winsorize_df(SIGMA_df, var_list=['SIGMA'])

In [ ]:
winsorized_SIGMA_df.to_csv('../Data/campbell_data/campbell_SIGMA.csv')

In [24]:
SIGMA_df1.head()

,PERMNO,year,month,is_non_zero,var,rolling_is_non_zero,rolling_var,SIGMA
0,10000,1986,1,11,0.004261,NaN,NaN,NaN
1,10000,1986,2,11,0.000961,NaN,NaN,NaN
2,10000,1986,3,9,0.001985,NaN,NaN,NaN
3,10000,1986,4,11,0.000126,31.0,0.007207,0.246047
4,10000,1986,5,17,0.001510,31.0,0.003072,0.160645
